# Get data

In [1]:
!git clone https://github.com/khumbuai/ulmfit_keras.git

Cloning into 'ulmfit_keras'...
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 536 (delta 0), reused 0 (delta 0), pack-reused 535
Receiving objects: 100% (536/536), 108.75 KiB | 2.22 MiB/s, done.
Resolving deltas: 100% (343/343), done.


In [0]:
import os
os.chdir('./ulmfit_keras/')

In [3]:
!chmod -R 777 *
!./get_data.sh

=== Acquiring datasets ===
---
- Downloading WikiText-2 (WT2)
- Downloading WikiText-103 (WT2)
--2019-05-08 10:22:52--  https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-v1.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.132.229
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.132.229|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 190229076 (181M) [application/zip]
Saving to: ‘wikitext-103-v1.zip’

wikitext-103-v1.zip 100%[===================>] 181.42M  78.5MB/s    in 2.3s    

2019-05-08 10:22:55 (78.5 MB/s) - ‘wikitext-103-v1.zip’ saved [190229076/190229076]

- Downloading enwik8 (Character)
--2019-05-08 10:23:02--  http://mattmahoney.net/dc/enwik8.zip
Resolving mattmahoney.net (mattmahoney.net)... 67.195.197.75
Connecting to mattmahoney.net (mattmahoney.net)|67.195.197.75|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36445475 (35M) [application/zip]
Saving to: ‘enwik8.zip’

enwik8.zip        

In [0]:
!mkdir keras_lm/assets

In [0]:
!cp -rf data/* keras_lm/assets

In [0]:
!rm -rf data

In [0]:
import os
os.chdir('./keras_lm')



## Datasets

## Pretrained models

In [8]:

!wget -P assets/wikitext-103 http://files.fast.ai/models/wt103/itos_wt103.pkl

--2019-05-08 10:23:43--  http://files.fast.ai/models/wt103/itos_wt103.pkl
Resolving files.fast.ai (files.fast.ai)... 67.205.15.147
Connecting to files.fast.ai (files.fast.ai)|67.205.15.147|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4161252 (4.0M) [text/plain]
Saving to: ‘assets/wikitext-103/itos_wt103.pkl’

itos_wt103.pkl      100%[===================>]   3.97M  18.4MB/s    in 0.2s    

2019-05-08 10:23:43 (18.4 MB/s) - ‘assets/wikitext-103/itos_wt103.pkl’ saved [4161252/4161252]



In [9]:
!wget -P assets/weights/ http://files.fast.ai/models/wt103/fwd_wt103.h5

--2019-05-08 10:23:44--  http://files.fast.ai/models/wt103/fwd_wt103.h5
Resolving files.fast.ai (files.fast.ai)... 67.205.15.147
Connecting to files.fast.ai (files.fast.ai)|67.205.15.147|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 462387687 (441M) [text/plain]
Saving to: ‘assets/weights/fwd_wt103.h5’

fwd_wt103.h5        100%[===================>] 440.97M  99.6MB/s    in 4.6s    

2019-05-08 10:23:49 (95.5 MB/s) - ‘assets/weights/fwd_wt103.h5’ saved [462387687/462387687]



In [0]:
#!wget -xP assets/weights/  http://files.fast.ai/models/wt103/fwd_wt103_enc.h5
#!wget -xP assets/weights/  http://files.fast.ai/models/wt103/bwd_wt103_enc.h5
#!wget -xP assets/weights/  http://files.fast.ai/models/wt103/bwd_wt103.h5

## Install packages

In [11]:
!pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
!pip install fastai

Looking in links: https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
     |████████████████████████████████| 576.2MB 22kB/s 


In [12]:
!pip install attrdict

# Prepare Wikitxt

In [13]:
!ls assets/wikitext-103/

itos_wt103.pkl	test.txt  train.txt  valid.txt


In [0]:
#from preprocessing.create_corpus import *
from utils.utils import LoadParameters

params = LoadParameters()
WIKI103_FOLDER = params.params['wiki103_text_folder']
PYTORCH_IDX2WORD_FILEPATH = params.params['pytorch_idx2word_filepath']
from utils.utils import LoadParameters

create_corpus(PYTORCH_IDX2WORD_FILEPATH, WIKI103_FOLDER)

1499551it [1:04:34, 737.55it/s]

# Train generic LM

In [6]:
from language_model.train import *


import pickle

import keras.backend as K
import numpy as np
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam

from language_model.batch_generators import BatchGenerator
from language_model.model import build_language_model


from utils.utils import LoadParameters

# 1. Load parameters from config.yaml file
params = LoadParameters()
WIKIPEDIA_CORPUS_FILE = params.params['wikipedia_corpus_file']
LANGUAGE_MODEL_WEIGHT = params.params['language_model_weight']
LANGUAGE_MODEL_PARAMS = params.params['lm_params']

epochs = params.params['lm_epochs']
batch_size = params.params['lm_batch_size']
valid_batch_size = params.params['lm_valid_batch_size']
seq_len = params.params['lm_seq_len']

# 2. Load Corpus
[train, valid, test, word2idx, idx2word] = pickle.load(open(WIKIPEDIA_CORPUS_FILE, 'rb'))

train_gen = BatchGenerator(train, batch_size, seq_len, modify_seq_len=True)
valid_gen = BatchGenerator(valid, valid_batch_size, seq_len, modify_seq_len=True)

K.clear_session()
num_words = len(word2idx) + 1

model = build_language_model(num_words, **LANGUAGE_MODEL_PARAMS)
model.compile(loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'],
              optimizer=Adam(lr=3e-4, beta_1=0.8, beta_2=0.99))

model.summary()

callbacks = [EarlyStopping(patience=5),
             ModelCheckpoint(LANGUAGE_MODEL_WEIGHT, save_weights_only=True)]

history = model.fit_generator(train_gen,
                              steps_per_epoch=len(train) // (seq_len * batch_size),
                              epochs=epochs,
                              validation_data=valid_gen,
                              validation_steps=len(valid) // (seq_len * batch_size),
                              callbacks=callbacks,
                              )

evaluate_model(model, word2idx, 'i feel sick and go to the ')



Using TensorFlow backend.


ModuleNotFoundError: ignored

In [9]:
import pickle

import keras.backend as K
import numpy as np
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam

from language_model.batch_generators import BatchGenerator
from language_model.model import build_language_model


def evaluate_model(model, word2idx, test_sentence, num_predictions=5):
    """
    Visual preidictions of the language model. The test_sentence is appended with num_predictions words,
    which are predicted as the next words from the model.
    :param str test_sentence:
    :param int num_predictions:
    :return: None
    """

    idx2word = {i: w for w, i in word2idx.items()}
    test_sentence = test_sentence.split()
    encoded_sentence = [word2idx[w] for w in test_sentence]

    for i in range(num_predictions):
        X = np.reshape(encoded_sentence, (1, len(encoded_sentence)))

        pred = model.predict(X)
        answer = np.argmax(pred, axis=2)

        predicted_idx = answer[0][-2]
        encoded_sentence.append(predicted_idx)

    print(' '.join([idx2word[i] for i in encoded_sentence]))


from utils import LoadParameters

# 1. Load parameters from config.yaml file
params = LoadParameters()
WIKIPEDIA_CORPUS_FILE = params.params['wikipedia_corpus_file']
LANGUAGE_MODEL_WEIGHT = params.params['language_model_weight']
LANGUAGE_MODEL_PARAMS = params.params['lm_params']

epochs = params.params['lm_epochs']
batch_size = params.params['lm_batch_size']
valid_batch_size = params.params['lm_valid_batch_size']
seq_len = params.params['lm_seq_len']

# 2. Load Corpus
[train, valid, test, word2idx, idx2word] = pickle.load(open(WIKIPEDIA_CORPUS_FILE, 'rb'))

train_gen = BatchGenerator(train, batch_size, seq_len, modify_seq_len=True)
valid_gen = BatchGenerator(valid, valid_batch_size, seq_len, modify_seq_len=True)

K.clear_session()
num_words = len(word2idx) + 1

model = build_language_model(num_words, **LANGUAGE_MODEL_PARAMS)
model.compile(loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'],
              optimizer=Adam(lr=3e-4, beta_1=0.8, beta_2=0.99))

model.summary()

callbacks = [EarlyStopping(patience=5),
             ModelCheckpoint(LANGUAGE_MODEL_WEIGHT, save_weights_only=True)]

history = model.fit_generator(train_gen,
                              steps_per_epoch=len(train) // (seq_len * batch_size),
                              epochs=epochs,
                              validation_data=valid_gen,
                              validation_steps=len(valid) // (seq_len * batch_size),
                              callbacks=callbacks,
                              )

evaluate_model(model, word2idx, 'i feel sick and go to the ')


ModuleNotFoundError: ignored

# Fine tune language model

In [0]:
from language_model.finetune_lm import *

import pandas as pd
from collections import Counter, defaultdict

from utils.utils import LoadParameters

# 1. Load parameters from config.yaml
params = LoadParameters()

PYTORCH_ITOS_FILEPATH = params.params['pytorch_idx2word_filepath']
WEIGTHS_FILEPATH = params.params['language_model_weight']
FINETUNED_WEIGTHS_FILEPATH = params.params['finetuned_language_model_weight']
FINETUNED_WORD2IDX_FILEPATH = params.params['finetuned_word2idx_filepath']
LANGUAGE_MODEL_PARAMS = params.params['lm_params']
FINETUNED_CORPUS_FILEPATH = params.params['finetuned_corpus_filepath']

batch_size = params.params['lm_batch_size']
seq_length = params.params['lm_seq_len']

# 2. Initialize pretrained language model.
K.clear_session()
with open(PYTORCH_ITOS_FILEPATH, 'rb') as f:
    words = pickle.load(f)

word2idx = {word: idx for idx, word in enumerate(words)}

word2idx = defaultdict(lambda: word2idx['_unk_'], word2idx)

num_words = len(word2idx)

language_model = build_language_model(num_words, **LANGUAGE_MODEL_PARAMS)
language_model.summary()
language_model.load_weights(WEIGTHS_FILEPATH)

# 3. Open target training dataset. We assume that the dataframes contains the already tokenized sentences.
train_df = pd.read_csv(os.path.join(FINETUNED_CORPUS_FILEPATH, 'train.csv'), names=['mood', 'text'])
valid_df = pd.read_csv(os.path.join(FINETUNED_CORPUS_FILEPATH, 'test.csv'), names=['mood', 'text'])

train_text = read_df(train_df)
valid_text = read_df(valid_df)

text = train_text + valid_text

unique_words = [o for o, c in Counter(text).most_common(100000) if c > 10]

# 4. Add new words to the word2idx dictionary and update the language model.
num_words_not_in_corpus = len(set(unique_words) - set(word2idx.keys()))
word2idx = update_word2idx(unique_words, word2idx)
language_model = update_language_model(language_model, num_words_not_in_corpus, **LANGUAGE_MODEL_PARAMS)

language_model.summary()

# 5. Prepare training and validation data
train = [word2idx[word] for word in train_text]
valid = [word2idx[word] for word in valid_text]

# 6. Finetune model
train_gen = BatchGenerator(train, batch_size, seq_length, modify_seq_len=True)
valid_gen = BatchGenerator(valid, batch_size, seq_length, modify_seq_len=True)

callbacks = [EarlyStopping(patience=5),
             ModelCheckpoint(FINETUNED_WEIGTHS_FILEPATH, save_weights_only=True)]

language_model.compile(loss=sparse_categorical_crossentropy,
                       metrics=['sparse_categorical_accuracy'],
                       optimizer=LRMultiplierSGD(lr=0.2 * 3e-4, momentum=0., decay=0., nesterov=False)
                       )

language_model.fit_generator(train_gen,
                             steps_per_epoch=len(train) // (seq_length * batch_size),
                             epochs=20,
                             validation_data=valid_gen,
                             validation_steps=len(valid) // (seq_length * batch_size),
                             callbacks=callbacks,
                             )

evaluate_model(language_model, word2idx, 'i feel sick and go to the', num_predictions=5)

# 7. Save word2idx dictionary
with open(FINETUNED_WORD2IDX_FILEPATH, 'wb') as f:
    pickle.dump(word2idx, f)

# ULMFiT integration (LM+Cls)

In [0]:
from trasnfer_model.train import *

from utils.utils import LoadParameters

# 1. Load parameters from config.yaml
params = LoadParameters()

CORPUS_FILEPATH = params.params['wikipedia_corpus_file']
TRAINING_DATA_FILEPATH = params.params['classification_csv']
CLASSIFICATON_MODEL_FILE = params.params['classifiaction_language_model_weight']

FINETUNED_WEIGTHS_FILEPATH = params.params['finetuned_language_model_weight']
NUMBER_OF_LABELS = params.params['number_of_labels']

# 2. Initialize pretrained language model.
K.clear_session()
wikitext_corpus = pickle.load(open(CORPUS_FILEPATH,'rb'))
num_words = len(wikitext_corpus.word2idx) +1

language_model = build_language_model(num_words, embedding_size=300, use_gpu=True)
language_model.summary()
language_model.load_weights(FINETUNED_WEIGTHS_FILEPATH)

# 3. Initialize classifiaction_model
classification_model = build_classification_model(language_model, NUMBER_OF_LABELS, **params.params['cm_params'])

# 4. Load X_train from pickle file
with open(TRAINING_DATA_FILEPATH, 'rb') as f:
    X_train, y_train = pickle.load(f)

# 5. Train classification model
epochs_list = [1 for layer in classification_model.layers]
learning_rates_list = [0.01 for layer in classification_model.layers]
train_classifiaction_model(classification_model, X_train, y_train, epochs_list, learning_rates_list)

# 6. Save classification model
classification_model.save(CLASSIFICATON_MODEL_FILE, overwrite=True)